<a href="https://colab.research.google.com/github/majagurdek/Machine-Learning/blob/main/lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn import datasets
data_breast_cancer = datasets.load_breast_cancer(as_frame=True)
X_bc = data_breast_cancer["data"][["mean texture", "mean symmetry"]]
y_bc = data_breast_cancer["target"]

In [ ]:
X_bc

,mean texture,mean symmetry
0,10.38,0.2419
1,17.77,0.1812
2,21.25,0.2069
3,20.38,0.2597
4,14.34,0.1809
...,...,...
564,22.39,0.1726
565,28.25,0.1752
566,28.08,0.1590
567,29.33,0.2397


In [ ]:
y_bc

0      0
1      0
2      0
3      0
4      0
      ..
564    0
565    0
566    0
567    0
568    1
Name: target, Length: 569, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_bc_train, X_bc_test, y_bc_train, y_bc_test = train_test_split(X_bc, y_bc, test_size=0.2)

In [ ]:
#budowanie ensembli

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
tree_clf = DecisionTreeClassifier()
log_clf = LogisticRegression(solver="lbfgs")
knn_clf = KNeighborsClassifier()

In [ ]:
voting_clf_hard = VotingClassifier(
    estimators=[('dt', tree_clf),
                ('lr', log_clf),
                ('kn', knn_clf)],
    voting='hard')
voting_clf_hard.fit(X_bc_train, y_bc_train)

VotingClassifier(estimators=[('dt', DecisionTreeClassifier()),
                             ('lr', LogisticRegression()),
                             ('kn', KNeighborsClassifier())])

In [ ]:
voting_clf_soft = VotingClassifier(
    estimators=[('dt', tree_clf),
                ('lr', log_clf),
                ('kn', knn_clf)],
    voting='soft')
voting_clf_soft.fit(X_bc_train, y_bc_train)

VotingClassifier(estimators=[('dt', DecisionTreeClassifier()),
                             ('lr', LogisticRegression()),
                             ('kn', KNeighborsClassifier())],
                 voting='soft')

In [ ]:
#accuracy

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
#zapisac jako liste par train, test

In [ ]:
list1 = []
list2 = []

In [ ]:
for clf in (tree_clf, log_clf, knn_clf, voting_clf_hard, voting_clf_soft):
    clf.fit(X_bc_train, y_bc_train)
    list2.append(clf)
    y_pred_train = clf.predict(X_bc_train)
    y_pred_test = clf.predict(X_bc_test)
    print(clf.__class__.__name__,accuracy_score(y_bc_train, y_pred_train), clf.__class__.__name__,
          accuracy_score(y_bc_test, y_pred_test))
    list1.append((accuracy_score(y_bc_train, y_pred_train), accuracy_score(y_bc_test, y_pred_test)))

DecisionTreeClassifier 1.0 DecisionTreeClassifier 0.7368421052631579
LogisticRegression 0.7142857142857143 LogisticRegression 0.7543859649122807
KNeighborsClassifier 0.756043956043956 KNeighborsClassifier 0.6754385964912281
VotingClassifier 0.8351648351648352 VotingClassifier 0.7017543859649122
VotingClassifier 0.9648351648351648 VotingClassifier 0.7192982456140351


In [ ]:
list1

[(1.0, 0.7368421052631579),
 (0.7142857142857143, 0.7543859649122807),
 (0.756043956043956, 0.6754385964912281),
 (0.8351648351648352, 0.7017543859649122),
 (0.9648351648351648, 0.7192982456140351)]

In [ ]:
import pickle

In [ ]:
with open('acc_vote.pkl', 'wb') as f:
    pickle.dump(list1, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('acc_vote.pkl', 'rb') as f:
    print(pickle.load(f))

[(1.0, 0.7368421052631579), (0.7142857142857143, 0.7543859649122807), (0.756043956043956, 0.6754385964912281), (0.8351648351648352, 0.7017543859649122), (0.9648351648351648, 0.7192982456140351)]


In [ ]:
list2

[DecisionTreeClassifier(),
 LogisticRegression(),
 KNeighborsClassifier(),
 VotingClassifier(estimators=[('dt', DecisionTreeClassifier()),
                              ('lr', LogisticRegression()),
                              ('kn', KNeighborsClassifier())]),
 VotingClassifier(estimators=[('dt', DecisionTreeClassifier()),
                              ('lr', LogisticRegression()),
                              ('kn', KNeighborsClassifier())],
                  voting='soft')]

In [ ]:
with open('vote.pkl', 'wb') as f:
    pickle.dump(list2, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('vote.pkl', 'rb') as f:
    print(pickle.load(f))

[DecisionTreeClassifier(), LogisticRegression(), KNeighborsClassifier(), VotingClassifier(estimators=[('dt', DecisionTreeClassifier()),
                             ('lr', LogisticRegression()),
                             ('kn', KNeighborsClassifier())]), VotingClassifier(estimators=[('dt', DecisionTreeClassifier()),
                             ('lr', LogisticRegression()),
                             ('kn', KNeighborsClassifier())],
                 voting='soft')]


In [ ]:
#bagging, pasting itp

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=30,max_samples=1.0, bootstrap=True)
bag_clf_half = BaggingClassifier(DecisionTreeClassifier(), n_estimators=30,max_samples=0.5, bootstrap=True)
pas_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=30,max_samples=1.0, bootstrap=False)
pas_clf_half = BaggingClassifier(DecisionTreeClassifier(), n_estimators=30,max_samples=0.5, bootstrap=False)
rnd_clf = RandomForestClassifier(n_estimators=30)
ada_clf = AdaBoostClassifier(n_estimators=30)
gb_clf = GradientBoostingClassifier(n_estimators=30)


In [ ]:
#accuracy

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
acc_bag = []
bag = []

In [ ]:
for clf in (bag_clf, bag_clf_half, pas_clf, pas_clf_half, rnd_clf, ada_clf, gb_clf):
  clf.fit(X_bc_train, y_bc_train)
  bag.append(clf)
  acc_bag.append((
        accuracy_score(y_bc_train, clf.predict(X_bc_train)),
        accuracy_score(y_bc_test, clf.predict(X_bc_test))
    ))

In [ ]:
acc_bag

[(1.0, 0.7368421052631579),
 (0.9164835164835164, 0.7280701754385965),
 (1.0, 0.6666666666666666),
 (0.9604395604395605, 0.7192982456140351),
 (1.0, 0.7719298245614035),
 (0.8087912087912088, 0.7631578947368421),
 (0.8241758241758241, 0.7631578947368421)]

In [ ]:
with open('acc_bag.pkl', 'wb') as f:
    pickle.dump(acc_bag, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('acc_bag.pkl', 'rb') as f:
    print(pickle.load(f))

[(1.0, 0.7368421052631579), (0.9164835164835164, 0.7280701754385965), (1.0, 0.6666666666666666), (0.9604395604395605, 0.7192982456140351), (1.0, 0.7719298245614035), (0.8087912087912088, 0.7631578947368421), (0.8241758241758241, 0.7631578947368421)]


In [ ]:
bag

[BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=30),
 BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=0.5,
                   n_estimators=30),
 BaggingClassifier(base_estimator=DecisionTreeClassifier(), bootstrap=False,
                   n_estimators=30),
 BaggingClassifier(base_estimator=DecisionTreeClassifier(), bootstrap=False,
                   max_samples=0.5, n_estimators=30),
 RandomForestClassifier(n_estimators=30),
 AdaBoostClassifier(n_estimators=30),
 GradientBoostingClassifier(n_estimators=30)]

In [ ]:
with open('bag.pkl', 'wb') as f:
    pickle.dump(bag, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('bag.pkl', 'rb') as f:
    print(pickle.load(f))

[BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=30), BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=0.5,
                  n_estimators=30), BaggingClassifier(base_estimator=DecisionTreeClassifier(), bootstrap=False,
                  n_estimators=30), BaggingClassifier(base_estimator=DecisionTreeClassifier(), bootstrap=False,
                  max_samples=0.5, n_estimators=30), RandomForestClassifier(n_estimators=30), AdaBoostClassifier(n_estimators=30), GradientBoostingClassifier(n_estimators=30)]


In [ ]:
#sampling cech

In [ ]:
X = data_breast_cancer["data"]
y = data_breast_cancer["target"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
acc_fea = []
fea = []

In [ ]:
smpl = BaggingClassifier(DecisionTreeClassifier(), n_estimators=30, max_features=2,  max_samples=0.5, bootstrap_features=False, bootstrap = True)

In [ ]:
smpl.fit(X_train, y_train)
fea.append(smpl)

In [ ]:
acc_fea.append((
        accuracy_score(y_train, smpl.predict(X_train)),
        accuracy_score(y_test, smpl.predict(X_test))))

In [ ]:
acc_fea

[(0.9978021978021978, 0.9385964912280702)]

In [ ]:
with open('acc_fea.pkl', 'wb') as f:
    pickle.dump(acc_fea, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('acc_fea.pkl', 'rb') as f:
    print(pickle.load(f))

[(0.9978021978021978, 0.9385964912280702)]


In [ ]:
with open('fea.pkl', 'wb') as f:
    pickle.dump(fea, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('fea.pkl', 'rb') as f:
    print(pickle.load(f))

[BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_features=2,
                  max_samples=0.5, n_estimators=30)]


In [ ]:
#data frame

In [ ]:
data = []

In [ ]:
for estimator, features in zip(smpl.estimators_, smpl.estimators_features_):
  y_pred_train2 = estimator.predict(X_train.iloc[:, features])
  y_pred_test2 = estimator.predict(X_test.iloc[:, features])
  data.append([accuracy_score(y_train, y_pred_train2), accuracy_score(y_test, y_pred_test2), list(X.columns[features])])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has featu

In [ ]:
acc_fea_rank = pd.DataFrame(data, columns = ["Train Accuracy", "Test Accuracy", "Features"])

In [ ]:
acc_fea_rank.sort_values(by=['Test Accuracy', 'Train Accuracy'], ascending=False, inplace=True)

In [ ]:
acc_fea_rank

,Train Accuracy,Test Accuracy,Features
28,0.934066,0.956140,"[mean concavity, worst area]"
0,0.934066,0.929825,"[compactness error, mean concavity]"
1,0.923077,0.921053,"[concavity error, worst perimeter]"
4,0.931868,0.912281,"[concavity error, worst area]"
8,0.931868,0.912281,"[compactness error, worst concave points]"
22,0.958242,0.903509,"[concave points error, worst area]"
5,0.927473,0.903509,"[mean perimeter, worst texture]"
15,0.907692,0.903509,"[mean fractal dimension, mean perimeter]"
13,0.940659,0.894737,"[worst radius, mean radius]"
23,0.920879,0.885965,"[radius error, worst radius]"


In [ ]:
with open('acc_fea_rank.pkl', 'wb') as f:
    pickle.dump(acc_fea_rank, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('acc_fea_rank.pkl', 'rb') as f:
    print(pickle.load(f))

    Train Accuracy  Test Accuracy  \
28        0.934066       0.956140   
0         0.934066       0.929825   
1         0.923077       0.921053   
4         0.931868       0.912281   
8         0.931868       0.912281   
22        0.958242       0.903509   
5         0.927473       0.903509   
15        0.907692       0.903509   
13        0.940659       0.894737   
23        0.920879       0.885965   
26        0.898901       0.885965   
9         0.925275       0.877193   
21        0.896703       0.859649   
20        0.892308       0.850877   
10        0.890110       0.850877   
12        0.918681       0.842105   
2         0.916484       0.842105   
16        0.920879       0.833333   
19        0.901099       0.824561   
24        0.927473       0.815789   
17        0.923077       0.815789   
6         0.839560       0.771930   
25        0.868132       0.754386   
29        0.778022       0.719298   
7         0.793407       0.701754   
11        0.800000       0.675439   
1